In [1]:
import numpy as np
import pandas as pd 
import re 
import gensim 
import json
import timeit
import networkx as nx
import matplotlib.pyplot as plt

## Read patents

In [3]:
ids = pd.read_csv("gp-query2-plus.csv",index_col='code',skipinitialspace=True)
print(ids.isnull().sum())
print('length is',len(ids))

id                                0
title                             0
assignee                         15
inventor/author                 383
priority date                   324
filing/creation date              6
publication date                  1
grant date                    13623
result link                       0
representative figure link    10002
code.1                            0
citations                         0
abstract                        796
class                          3384
dtype: int64
length is 22573


In [4]:
ids.head()

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link,code.1,citations,abstract,class
code,,,,,,,,,,,,,,
KR100866792B1,KR-100866792-B1,Method and apparatus for generating face descr...,삼성전자주식회사,"문영수, 박규태, 자오지아리, 황산성, 황원준",2007-01-10,2007-01-10,2008-11-04,2008-11-04,https://patents.google.com/patent/KR100866792B...,https://patentimages.storage.googleapis.com/b5...,KR100866792B1,"['US20150178547A1', 'KR100707195B1', 'EP171651...",\n\n 본 발명은 확장 국부 이진 패턴(Extended Local Binary P...,G06K9/
KR100703693B1,KR-100703693-B1,System and method for face recognition,삼성전자주식회사,"김정배, 이종하",2005-01-13,2005-01-13,2007-04-05,2007-04-05,https://patents.google.com/patent/KR100703693B...,https://patentimages.storage.googleapis.com/5a...,KR100703693B1,"['US10728242B2', 'US8433922B2', 'US9122913B2',...","\n\n 본 발명은 얼굴 인식에 관한 것으로서, 본 발명의 실시에 따른 얼굴 인식 ...",G06K9/
EP1732028B1,EP-1732028-B1,System and method for detecting an eye,"Delphi Technologies, Inc.","Riad I. Hammoud, Andrew L. Wilhelm",2005-06-10,2006-05-31,2012-01-11,2012-01-11,https://patents.google.com/patent/EP1732028B1/en,https://patentimages.storage.googleapis.com/88...,EP1732028B1,"['EP0989517B1', 'US7206435B2', 'US6028949A', '...",NaN,G06K9/
US9396319B2,US-9396319-B2,Method of criminal profiling and person identi...,"Laird H. Shuart, Marcia L Shuart, Sharon E Jan...","Laird H. Shuart, Marcia L Shuart, Sharon E Jan...",2013-09-30,2014-09-24,2016-07-19,2016-07-19,https://patents.google.com/patent/US9396319B2/en,https://patentimages.storage.googleapis.com/ff...,US9396319B2,[],\nA method of criminal profiling and person id...,G06F21
KR100846500B1,KR-100846500-B1,Method and apparatus for recognizing face usin...,삼성전자주식회사,"기석철, 문영수, 박규태, 이종하, 황산성, 황원준",2006-11-08,2006-11-08,2008-07-17,2008-07-17,https://patents.google.com/patent/KR100846500B...,https://patentimages.storage.googleapis.com/c9...,KR100846500B1,"['US10565433B2', 'US20110135166A1']",\n\n 본 발명은 확장된 가보 웨이브렛 특징 들(Gabor Wavelet Feat...,G06K9/


In [7]:
ids = ids[ids.abstract.notna()]
ids["class"].fillna(value="NA",inplace = True)
ids['class'] = ids['class'].astype('str').apply(lambda x: x[:3] if len(x) >= 3 else 'NA')
ids["class"] = ids["class"].apply(lambda x: 'NA' if x.find('G06')<0 and x.find('H04')<0 else x)
ids['abstract'] = ids['abstract'].str.replace(r'[^\x00-\x7F]+', '')

print(ids.isnull().sum())
ids['class'].value_counts()

id                                0
title                             0
assignee                          5
inventor/author                 369
priority date                   315
filing/creation date              1
publication date                  0
grant date                    13565
result link                       0
representative figure link     9736
code.1                            0
citations                         0
abstract                          0
class                             0
dtype: int64


NA    21777
Name: class, dtype: int64

## Read articles

In [11]:
arts = pd.read_csv("q2_topics.csv",skipinitialspace=True)
arts

,EID,Year,Cited by,Link,Abstract,EID.1,topic
0,85099418104,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Generally, the decision rule for classifying u...",85099418104,6
1,85099378785,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,This survey explores how Deep Learning has bat...,85099378785,6
2,85094573545,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,The way electorates were influenced to vote fo...,85094573545,7
3,85100650454,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,This study is informed by two research gaps. O...,85100650454,0
4,85101111750,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Over recent years, the development of online s...",85101111750,0
5,85099183156,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,Popular social media networks provide the perf...,85099183156,6
6,85100059208,2021,1.0,https://www.scopus.com/inward/record.uri?eid=2...,"With the advent of smart phones, the popularit...",85100059208,1
7,85100273731,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Today, Android stands out amongst the most wel...",85100273731,0
8,85098453070,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,Social media is used as a dominant source of n...,85098453070,6
9,85099261308,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,Zero-day malware samples pose a considerable d...,85099261308,6


# Topic Modeling

In [16]:
x_train = [abstract for abstract in ids.abstract]
x_test = [abstract for abstract in arts.Abstract]

print(len(x_train),len(x_test))
#x_train = x_train.extend(x_test)






21777 7405


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

it_stopwords = stopwords.words('english')

tf_vectorizer = CountVectorizer(stop_words=it_stopwords, min_df=5,max_features = 1000, ngram_range=(1,2))
tf = tf_vectorizer.fit_transform(x_train)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bonin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 8

lda = LatentDirichletAllocation(n_components=n_components, max_iter=10,
                                learning_method = 'batch',
                                n_jobs=-1,verbose=1)
lda.fit(tf)


iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(n_components=8, n_jobs=-1, verbose=1)

In [24]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = f'Topic {topic_idx}: '
        message += ', '.join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [33]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic 0: unit, machine, means, virtual, said, signal, voice, domain, name, first, audio, die, der, characterized, speech, 10, device, translation, second, und
Topic 1: first, data, set, second, based, file, one, method, content, plurality, system, includes, event, key, user, associated, using, search, model, machine
Topic 2: information, data, user, terminal, server, method, invention, service, system, management, present, according, present invention, authentication, based, mobile, application, comprises, storage, device
Topic 3: data, device, may, network, user, system, computer, security, one, computing, systems, based, methods, application, include, devices, access, associated, method, disclosed
Topic 4: control, device, system, communication, vehicle, connected, invention, wireless, power, mobile, unit, provided, monitoring, sensor, network, display, body, camera, end, equipment
Topic 5: method, data, invention, model, network, detection, image, target, feature, based, according, 

In [27]:
doc_topics = lda.transform(tf)
doc_topics

array([[0.0027218 , 0.00272178, 0.12116603, ..., 0.8625038 , 0.00272409,
        0.00272143],
       [0.00178717, 0.00178793, 0.39289555, ..., 0.44457467, 0.15359187,
        0.00178643],
       [0.0043144 , 0.00431496, 0.00431795, ..., 0.00431654, 0.00432323,
        0.18244929],
       ...,
       [0.0172772 , 0.07226136, 0.1801852 , ..., 0.29539401, 0.00164583,
        0.2416593 ],
       [0.05997355, 0.00115988, 0.08250345, ..., 0.0011586 , 0.35530427,
        0.27490183],
       [0.56043166, 0.0026663 , 0.00266416, ..., 0.0026652 , 0.1206485 ,
        0.00266548]])

In [28]:
ids['topic'] = np.argmax(doc_topics,axis=1)

In [29]:
ids['topic'].value_counts()

3    5428
5    3182
2    3098
1    3045
4    2375
6    2243
7    1333
0    1073
Name: topic, dtype: int64

In [30]:
tf2 = tf_vectorizer.transform(x_test)
arts_topics = lda.transform(tf2)


In [31]:
arts['topic'] = np.argmax(arts_topics,axis=1)
arts['topic'].value_counts()

5    3034
1    1838
7    1074
3     928
2     254
4     204
0      61
6      12
Name: topic, dtype: int64

In [ ]:
ids.to_csv('../patents/q2_topics.csv')
arts.to_csv('../papers/q2_topics.csv')

# Nuovo metodo

In [38]:
# Load the regular expression library
import re# Remove punctuation
ids['abstract_processed'] = ids['abstract'].map(lambda x: re.sub('[,\.!?]', '', x))# Convert the titles to lowercase
ids['abstract_processed'] = ids['abstract_processed'].map(lambda x: x.lower())# Print out the first rows of papers
ids['abstract'].head()

code
KR100866792B1    \n\n      (Extended Local Binary Pattern)     ...
KR100703693B1    \n\n      ,             ,        ,            ...
US9396319B2      \nA method of criminal profiling and person id...
KR100846500B1    \n\n       (Gabor Wavelet Features)        ,  ...
US9847974B2      \nDisclosed are devices and methods for proces...
Name: abstract, dtype: object

In [100]:
ids.head()

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link,code.1,citations,abstract,class,topic,abstract_processed
code,,,,,,,,,,,,,,,,
KR100866792B1,KR-100866792-B1,Method and apparatus for generating face descr...,삼성전자주식회사,"문영수, 박규태, 자오지아리, 황산성, 황원준",2007-01-10,2007-01-10,2008-11-04,2008-11-04,https://patents.google.com/patent/KR100866792B...,https://patentimages.storage.googleapis.com/b5...,KR100866792B1,"['US20150178547A1', 'KR100707195B1', 'EP171651...",\n\n (Extended Local Binary Pattern) ...,NA,5,\n\n (extended local binary pattern) ...
KR100703693B1,KR-100703693-B1,System and method for face recognition,삼성전자주식회사,"김정배, 이종하",2005-01-13,2005-01-13,2007-04-05,2007-04-05,https://patents.google.com/patent/KR100703693B...,https://patentimages.storage.googleapis.com/5a...,KR100703693B1,"['US10728242B2', 'US8433922B2', 'US9122913B2',...","\n\n , , , ...",NA,5,\n\n ...
US9396319B2,US-9396319-B2,Method of criminal profiling and person identi...,"Laird H. Shuart, Marcia L Shuart, Sharon E Jan...","Laird H. Shuart, Marcia L Shuart, Sharon E Jan...",2013-09-30,2014-09-24,2016-07-19,2016-07-19,https://patents.google.com/patent/US9396319B2/en,https://patentimages.storage.googleapis.com/ff...,US9396319B2,[],\nA method of criminal profiling and person id...,NA,3,\na method of criminal profiling and person id...
KR100846500B1,KR-100846500-B1,Method and apparatus for recognizing face usin...,삼성전자주식회사,"기석철, 문영수, 박규태, 이종하, 황산성, 황원준",2006-11-08,2006-11-08,2008-07-17,2008-07-17,https://patents.google.com/patent/KR100846500B...,https://patentimages.storage.googleapis.com/c9...,KR100846500B1,"['US10565433B2', 'US20110135166A1']","\n\n (Gabor Wavelet Features) , ...",NA,5,\n\n (gabor wavelet features) ...
US9847974B2,US-9847974-B2,Image document processing in a client-server s...,Xerox Corporation,Pramod Sankar Kompalli,2016-04-28,2016-04-28,2017-12-19,2017-12-19,https://patents.google.com/patent/US9847974B2/en,https://patentimages.storage.googleapis.com/26...,US9847974B2,"['US9767381B2', 'US10410353B2', 'US20170243317...",\nDisclosed are devices and methods for proces...,NA,3,\ndisclosed are devices and methods for proces...


In [46]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = ids.abstract_processed.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['extended', 'local', 'binary', 'pattern', 'lbp', 'lbp', 'lbp', 'lbp', 'the', 'present', 'invention', 'discloses', 'method', 'and', 'apparatus', 'for', 'generating', 'facial', 'descriptor', 'using', 'an', 'extended', 'local', 'binary', 'pattern', 'and', 'method', 'and', 'apparatus', 'for']


In [47]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [53]:
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [71]:
import spacy# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
data_lemmatized = data_words_bigrams #in realtà è senza lemmatiazion

In [72]:
import gensim.corpora as corpora# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)# Create Corpus
texts = data_lemmatized# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 5), (7, 3), (8, 1), (9, 4), (10, 1), (11, 2), (12, 2), (13, 8), (14, 1), (15, 2), (16, 2), (17, 1), (18, 1), (19, 2), (20, 1), (21, 2), (22, 1), (23, 1), (24, 3), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1)]]


In [74]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [75]:
from pprint import pprint# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.027*"text" + 0.018*"document" + 0.017*"voice" + 0.016*"knowledge" + '
  '0.014*"word" + 0.014*"propaganda" + 0.013*"de" + 0.012*"speech" + '
  '0.012*"character" + 0.011*"audio"'),
 (1,
  '0.011*"power" + 0.010*"provided" + 0.009*"one" + 0.009*"machine" + '
  '0.009*"body" + 0.009*"two" + 0.008*"side" + 0.008*"utility" + '
  '0.008*"arranged" + 0.007*"end"'),
 (2,
  '0.048*"image" + 0.037*"target" + 0.026*"method" + 0.024*"object" + '
  '0.019*"video" + 0.017*"invention" + 0.013*"according" + 0.012*"information" '
  '+ 0.010*"corresponding" + 0.009*"comprises"'),
 (3,
  '0.029*"may" + 0.020*"system" + 0.017*"based" + 0.016*"computer" + '
  '0.015*"systems" + 0.015*"security" + 0.015*"one" + 0.014*"computing" + '
  '0.013*"methods" + 0.011*"include"'),
 (4,
  '0.050*"data" + 0.044*"one" + 0.041*"user" + 0.030*"least" + 0.023*"content" '
  '+ 0.019*"device" + 0.016*"includes" + 0.014*"system" + 0.014*"plurality" + '
  '0.013*"method"'),
 (5,
  '0.038*"data" + 0.033*"system" + 0

In [76]:
from gensim.models import CoherenceModel# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4912679349772316


In [85]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=5, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [86]:
from pprint import pprint# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.030*"data" + 0.023*"method" + 0.017*"invention" + 0.015*"information" + '
  '0.012*"target" + 0.012*"model" + 0.012*"according" + 0.010*"based" + '
  '0.008*"detection" + 0.008*"comprises"'),
 (1,
  '0.016*"control" + 0.016*"system" + 0.015*"module" + 0.011*"vehicle" + '
  '0.010*"device" + 0.007*"unit" + 0.006*"monitoring" + 0.006*"invention" + '
  '0.006*"provided" + 0.006*"connected"'),
 (2,
  '0.045*"image" + 0.014*"signal" + 0.013*"object" + 0.013*"first" + '
  '0.011*"second" + 0.009*"unit" + 0.008*"one" + 0.007*"least" + '
  '0.007*"device" + 0.007*"sensor"'),
 (3,
  '0.027*"data" + 0.023*"one" + 0.017*"system" + 0.016*"network" + 0.016*"may" '
  '+ 0.014*"based" + 0.011*"least" + 0.011*"device" + 0.011*"computer" + '
  '0.009*"first"'),
 (4,
  '0.044*"information" + 0.037*"user" + 0.023*"device" + 0.018*"data" + '
  '0.018*"first" + 0.017*"server" + 0.013*"system" + 0.013*"terminal" + '
  '0.013*"service" + 0.010*"content"')]


In [87]:
from gensim.models import CoherenceModel# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4645070143247148


In [88]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=8, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [89]:
from pprint import pprint# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.045*"data" + 0.026*"method" + 0.022*"invention" + 0.015*"according" + '
  '0.014*"target" + 0.011*"module" + 0.011*"comprises" + 0.010*"discloses" + '
  '0.010*"model" + 0.010*"based"'),
 (1,
  '0.021*"control" + 0.014*"vehicle" + 0.014*"system" + 0.013*"device" + '
  '0.012*"module" + 0.008*"connected" + 0.008*"provided" + 0.008*"monitoring" '
  '+ 0.007*"power" + 0.007*"machine"'),
 (2,
  '0.042*"image" + 0.019*"object" + 0.014*"unit" + 0.014*"signal" + '
  '0.009*"invention" + 0.008*"method" + 0.008*"two" + 0.008*"area" + '
  '0.007*"recognition" + 0.007*"position"'),
 (3,
  '0.023*"data" + 0.021*"one" + 0.020*"system" + 0.020*"may" + 0.015*"based" + '
  '0.013*"security" + 0.012*"computer" + 0.011*"systems" + 0.009*"methods" + '
  '0.008*"computing"'),
 (4,
  '0.047*"device" + 0.040*"data" + 0.034*"first" + 0.024*"user" + 0.022*"one" '
  '+ 0.020*"second" + 0.016*"least" + 0.013*"system" + 0.013*"method" + '
  '0.012*"includes"'),
 (5,
  '0.076*"information" + 0.027*"user

In [90]:
from gensim.models import CoherenceModel# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.47313326610615264


In [91]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=15, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [92]:
from pprint import pprint# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.045*"text" + 0.034*"voice" + 0.030*"document" + 0.028*"audio" + '
  '0.024*"candidate" + 0.023*"word" + 0.022*"knowledge" + 0.019*"speech" + '
  '0.019*"information" + 0.017*"character"'),
 (1,
  '0.030*"device" + 0.025*"machine" + 0.020*"virtual" + 0.020*"power" + '
  '0.017*"body" + 0.016*"provided" + 0.014*"side" + 0.013*"arranged" + '
  '0.013*"utility" + 0.013*"two"'),
 (2,
  '0.102*"image" + 0.059*"target" + 0.047*"object" + 0.036*"video" + '
  '0.018*"information" + 0.016*"method" + 0.016*"area" + 0.014*"camera" + '
  '0.014*"images" + 0.014*"picture"'),
 (3,
  '0.044*"may" + 0.024*"computer" + 0.023*"systems" + 0.021*"computing" + '
  '0.020*"application" + 0.020*"system" + 0.020*"methods" + 0.018*"device" + '
  '0.017*"based" + 0.017*"include"'),
 (4,
  '0.106*"data" + 0.065*"one" + 0.041*"least" + 0.028*"content" + '
  '0.023*"includes" + 0.022*"plurality" + 0.021*"system" + 0.019*"device" + '
  '0.017*"based" + 0.016*"user"'),
 (5,
  '0.101*"information" + 0.051*"u

In [93]:
from gensim.models import CoherenceModel# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4805367024374942


In [94]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       per_word_topics=True)

In [95]:
from pprint import pprint# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.012*"information" + 0.009*"method" + 0.009*"network" + 0.009*"invention" '
  '+ 0.009*"first" + 0.008*"content" + 0.008*"communication" + 0.007*"data" + '
  '0.007*"system" + 0.007*"based"'),
 (1,
  '0.019*"device" + 0.018*"system" + 0.008*"data" + 0.008*"method" + '
  '0.008*"control" + 0.006*"display" + 0.005*"user" + 0.005*"one" + '
  '0.005*"based" + 0.005*"computer"'),
 (2,
  '0.016*"image" + 0.013*"method" + 0.011*"invention" + 0.010*"information" + '
  '0.008*"object" + 0.007*"system" + 0.007*"data" + 0.007*"key" + 0.006*"one" '
  '+ 0.006*"user"'),
 (3,
  '0.034*"data" + 0.016*"one" + 0.015*"system" + 0.013*"method" + '
  '0.012*"device" + 0.012*"user" + 0.011*"based" + 0.010*"computer" + '
  '0.009*"may" + 0.009*"least"'),
 (4,
  '0.031*"data" + 0.018*"user" + 0.013*"one" + 0.013*"system" + 0.011*"may" + '
  '0.010*"first" + 0.009*"network" + 0.008*"device" + 0.007*"information" + '
  '0.007*"second"'),
 (5,
  '0.024*"data" + 0.018*"device" + 0.014*"information" + 0.

In [96]:
from gensim.models import CoherenceModel# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4064528063209588


In [97]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=200,
                                       chunksize=200,
                                       passes=10,
                                       per_word_topics=True)

In [98]:
from pprint import pprint# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.043*"image" + 0.012*"two" + 0.010*"body" + 0.009*"object" + '
  '0.008*"camera" + 0.008*"machine" + 0.007*"provided" + 0.007*"area" + '
  '0.007*"display" + 0.007*"frame"'),
 (1,
  '0.031*"system" + 0.025*"may" + 0.020*"security" + 0.018*"event" + '
  '0.015*"data" + 0.014*"based" + 0.013*"risk" + 0.013*"systems" + '
  '0.012*"time" + 0.010*"methods"'),
 (2,
  '0.036*"one" + 0.028*"data" + 0.027*"content" + 0.021*"least" + 0.019*"user" '
  '+ 0.017*"method" + 0.017*"set" + 0.016*"plurality" + 0.015*"based" + '
  '0.012*"includes"'),
 (3,
  '0.055*"application" + 0.050*"code" + 0.046*"file" + 0.032*"malicious" + '
  '0.032*"software" + 0.028*"computer" + 0.023*"program" + 0.022*"method" + '
  '0.015*"malware" + 0.012*"disclosed"'),
 (4,
  '0.051*"module" + 0.029*"system" + 0.026*"data" + 0.019*"invention" + '
  '0.015*"video" + 0.015*"intelligent" + 0.014*"monitoring" + 0.014*"platform" '
  '+ 0.014*"equipment" + 0.013*"management"'),
 (5,
  '0.076*"information" + 0.051*"user"

In [99]:
from gensim.models import CoherenceModel# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5068621153079309


In [102]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

  0%|                                                                                        | 0/540 [1:13:50<?, ?it/s]
C:\Users\bonin\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.099925 -0.132567       1        1  15.577745
5      0.004873  0.017901       2        1  13.380069
7     -0.145942 -0.007751       3        1  12.308065
2     -0.132553 -0.014560       4        1  11.489157
1     -0.070719 -0.182481       5        1  10.188711
6      0.098382  0.136735       6        1   9.873144
0      0.235869  0.160141       7        1   8.981969
4      0.247565 -0.147121       8        1   8.461734
3     -0.203940 -0.071501       9        1   4.999207
8     -0.133459  0.241203      10        1   4.740208, topic_info=      Category          Freq         Term         Total  loglift  logprob
term                                                                     
293    Default  13252.000000        first  13252.000000  30.0000  30.0000
45     Default  23930.000000  information  23930.000000  29.0000  29.0000
303    Default   7798.000000       second   7798.000000  28.0000  28.0000
54     Default  16109.000000         user  16109.000000  27.0000  27.0000
140    Default  13559.000000      network  13559.000000  26.0000  26.0000
322    Default   7646.000000       module   7646.000000  25.0000  25.0000
43     Default   6926.000000        image   6926.000000  24.0000  24.0000
472    Default   7303.000000  application   7303.000000  23.0000  23.0000
282    Default   6547.000000         unit   6547.000000  22.0000  22.0000
129    Default  19800.000000       device  19800.000000  21.0000  21.0000
1111   Default   4479.000000         code   4479.000000  20.0000  20.0000
211    Default  33645.000000         data  33645.000000  19.0000  19.0000
439    Default   4057.000000         file   4057.000000  18.0000  18.0000
837    Default   5910.000000      content   5910.000000  17.0000  17.0000
221    Default  16751.000000          one  16751.000000  16.0000  16.0000
274    Default   7056.000000      control   7056.000000  15.0000  15.0000
79     Default   8662.000000          may   8662.000000  14.0000  14.0000
87     Default  21942.000000       system  21942.000000  13.0000  13.0000
1843   Default   4073.000000      vehicle   4073.000000  12.0000  12.0000
16     Default  19307.000000       method  19307.000000  11.0000  11.0000
12     Default  11266.000000    invention  11266.000000  10.0000  10.0000
85     Default   3024.000000     software   3024.000000   9.0000   9.0000
64     Default   7174.000000     computer   7174.000000   8.0000   8.0000
683    Default   2818.000000    malicious   2818.000000   7.0000   7.0000
1456   Default   2870.000000          key   2870.000000   6.0000   6.0000
154    Default   5454.000000      service   5454.000000   5.0000   5.0000
264    Default   5627.000000     terminal   5627.000000   4.0000   4.0000
793    Default   3385.000000       signal   3385.000000   3.0000   3.0000
216    Default   8697.000000        least   8697.000000   2.0000   2.0000
245    Default   3282.000000        event   3282.000000   1.0000   1.0000
...        ...           ...          ...           ...      ...      ...
342    Topic10    141.752090          auf    142.666626   3.0427  -6.3905
2310   Topic10    137.885284        einen    138.799820   3.0425  -6.4182
4452   Topic10    128.742813         oder    129.657349   3.0420  -6.4868
3029   Topic10    125.870003           um    126.784599   3.0418  -6.5094
4340   Topic10    125.318871   decryption    126.233505   3.0418  -6.5138
10482  Topic10    124.901031        title    125.815811   3.0418  -6.5171
2213   Topic10    242.457764       secret    246.230682   3.0336  -5.8538
1456   Topic10   2687.882080          key   2870.634766   2.9833  -3.4481
3412   Topic10    740.821533   encryption    777.612915   3.0006  -4.7369
1917   Topic10    789.390198    encrypted    898.300537   2.9198  -4.6734
2471   Topic10    451.326294         hash    569.457092   2.8166  -5.2324
4205   Topic10    190.908707         keys    205.2017